# Logistic Regression

**import libraries**

In [53]:
from IPython.display import display

import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go

**Train-validate-test**

Some general definitions are:

- training dataset: the sample of data used to fit the model
- validation dataset: the sample of data used to evaluate the model and possibly to adjust the hyperparameters
- testing dataset the sample of data used for final model testing; not to be used for anything other than testing so that the result is unbiased

### Modeling with iris dataset

In [3]:
iris = sns.load_dataset("iris")
display(iris.head())

x = iris['petal_width']
X = x[:, np.newaxis]
y = iris['petal_length']

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


**Creating Test Dataset**

In [6]:
# Create the "remaining" and test datasets
X_remain, X_test, y_remain, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

**Creating training and validation dataset**

In [7]:
# Create the train and validation datasets

X_train, X_val, y_train, y_val = train_test_split(
    X_remain, y_remain, test_size=0.25, random_state=42)

# Print out sizes of train, validate, test datasets

print('Training data set samples:', len(X_train))
print('Validation data set samples:', len(X_val))
print('Test data set samples:', len(X_test))

Training data set samples: 90
Validation data set samples: 30
Test data set samples: 30


**Fitting the model and validating**

In [10]:
# Instantiate the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Use the VALIDATION set for prediction
y_predict = model.predict(X_val)

# Calculate the accuracy score
r2_score(y_val, y_predict)

0.9589442606386026

**Using the test data**

In [11]:
# Use the TEST set for prediction
y_predict_test = model.predict(X_test)

# Calculate the accuracy score

r2_score(y_test, y_predict_test)

0.9287783612248339

### Exploring geyser dataset with classification

**determining baseline**

In [12]:
geyser = sns.load_dataset("geyser")
display(geyser.head())

display(geyser.describe())

,duration,waiting,kind
0,3.600,79,long
1,1.800,54,short
2,3.333,74,long
3,2.283,62,short
4,4.533,85,long


,duration,waiting
count,272.000000,272.000000
mean,3.487783,70.897059
std,1.141371,13.594974
min,1.600000,43.000000
25%,2.162750,58.000000
50%,4.000000,76.000000
75%,4.454250,82.000000
max,5.100000,96.000000


**Setting baseline**

at the moment if we were to always guess long we would be right 63% of the time

In [14]:
# Find the number of counts for each type of eruption
geyser['kind'].value_counts(normalize=True)

long     0.632353
short    0.367647
Name: kind, dtype: float64

**looking at iris and penguin baselines**

- for the penguins data set if we were to always guess Adelie we would be right 44% of the time

- for the iris datasets regardless of what we guess we will be right 33% of the time

pretty poor baselines in this case...

In [17]:
penguin = sns.load_dataset('penguins')
iris = sns.load_dataset('iris')

print(penguin['species'].value_counts(normalize=True))
print(iris['species'].value_counts(normalize=True))

Adelie       0.441860
Gentoo       0.360465
Chinstrap    0.197674
Name: species, dtype: float64
setosa        0.333333
virginica     0.333333
versicolor    0.333333
Name: species, dtype: float64


## Logistic Regression

A logistic regression classifier is based on the sigmoid function which is an s-shaped curve. The equation for the sigmoid is given by:

$\sigma(x) = \frac{1}{1+exp(-x)}$

In [18]:
def sigmoid(x):
    return 1 / (1 + np.e**(-x))

In [25]:
# Plot the function
x_plot = np.linspace(-10, 10, 100)
sig_y = sigmoid(x_plot)

fig = px.line(x=x_plot, y=sig_y, title='Sigmoid function', width=600, labels={
                     'y': "sigma(x)",
                     "x": "x"
                 })
fig.show()

Over most of the range of the sigmoid function, the value is either 0 or 1, which is why this function is particularly suitable for a binary classifier. When we are fitting a model, we would like to find the coefficients that best fit the data. Including these coefficients results in an equation of this form:

$P(y_i = 1) = \frac{1}{1+e^{-(\beta_0+\beta_1x)}}$

where

$P(y_i=1)$
is the probability of observationibeing in class 1. The coefficients

$\beta_0$ and $\beta_1$ 

determine the shape of the function and are what we are trying to fit when we model our data. When we know the coefficients, we can make a prediction of the class for an observation $x$.

### Logistic regression and gesure dataset

In [27]:
geyser = sns.load_dataset("geyser")

# Choose one feature - we'll use the duration
x = geyser['duration']

# Import the label encoder and encode the 'kind' column
le = LabelEncoder()

# Create a new column with 0=long and 1=short class labels
geyser['kind_binary'] = le.fit_transform(geyser['kind'])
display(geyser.head())

# Assign the target variable to y
y = geyser['kind_binary']

,duration,waiting,kind,kind_binary
0,3.600,79,long,0
1,1.800,54,short,1
2,3.333,74,long,0
3,2.283,62,short,1
4,4.533,85,long,0


In [31]:
fig = px.scatter(geyser, x=x, y=y, width=800)
fig.show()

In [38]:
# Assign coefficient from previously fit model
beta_0 = 11.32
beta_1 = -3.65

# Define the sigmoid with the coefficients
def sigmoid_beta(x, beta_0, beta_1):
    exp = beta_0 + beta_1*x
    return 1 / (1 + np.e**(-exp))

x_model_plot = np.linspace(1, 6, 100)
y_model = sigmoid_beta(x_model_plot, beta_0, beta_1)

fig = px.scatter(geyser, x=x, y=y, width=800, title='Geyser eruption with model', labels={
                     "kind_binary": "P( y=1 )",
                     "x": "x (geyser duration - minutes)"
                 })
fig.add_trace(go.Scatter(x=x_model_plot, y=y_model,
                    mode='lines',
                    name='lines'))
fig.show()

Now let's use our function with the model parameters to make and interpret a prediction. We'll pretend that we have visited the geyser site and viewed an eruption, which we timed to be 3.25 minutes. Which class would this eruption belong to and with what probability?

We know from the equation above that the probability is for the observation to belong to class=1 (short eruption). Plugging in the values for x (3.25 minutes) along with the coefficients gives us the following equation:

 $P(y_i = 1) = \frac{1}{1+e^{-(\beta_0+\beta_1x)}}$ 

 $P(y_i = 1 \text{ when }x=3.25) = \frac{1}{1+e^{-(11.32-3.65\times3.25)}}$

In [39]:
# Plug in the above values
sigmoid_beta(3.25, beta_0, beta_1)

0.3676062104304375

We interpret this result to mean that the probability of belonging to class=1 (short) is 37%. The probability of the observation belonging to class=0 (long) is 100%-37% = 63%. Our model predicts that an eruption with a duration of 3.25 minutes would belong to the long class of eruption.

**plotting the waiting feature on top of data**

the next step is to actually start using logistic regression

In [45]:
fig = px.scatter(geyser, x=geyser['waiting'], y=y, width=800, title='Geyser eruption with model', labels={
                     "kind_binary": "P( y=1 )",
                     "waiting": "x (geyser waiting)"
                 })
fig.show()

### Predicting with logistic regression

In [50]:
df = sns.load_dataset('geyser')

le = LabelEncoder()

df['kind_binary'] = le.fit_transform(df['kind'])
display(df.head())

,duration,waiting,kind,kind_binary
0,3.600,79,long,0
1,1.800,54,short,1
2,3.333,74,long,0
3,2.283,62,short,1
4,4.533,85,long,0


**creating targets and features fitting model**

In [51]:
x = df['duration']
X = x[:, np.newaxis]

y = df['kind']

# Fitting model
model = LogisticRegression()
model.fit(X,y)

LogisticRegression()

**Using cross validation to test our model**

this is also why its important to have test data that isn't touched till the end

In [54]:
# Implement a cross-validation with k=5
print(cross_val_score(model, X, y, cv=5))

# Calculate the mean of the cross-validation scores
score_mean = cross_val_score(model, X, y, cv=5).mean()
print('The mean CV score is: ', score_mean)

[0.94545455 1.         1.         0.94444444 1.        ]
The mean CV score is:  0.977979797979798


In [58]:
# Feature matrix
features = ['duration', 'waiting']
X = df[features]

model = LogisticRegression()
model.fit(X, y)

print(cross_val_score(model, X, y, cv=5))
score_mean = cross_val_score(model, X, y, cv=5).mean()
print('The mean CV score is (two features): ', score_mean)

[1. 1. 1. 1. 1.]
The mean CV score is (two features):  1.0


The accuracy is perfect for this model. This is likely because the two classes have a very clear division. It's important to remember that not all data sets will be so easy to model with such accurate results!

**plotting geyser features for analysis**

In [66]:
fig = px.scatter(df, x='duration', y='waiting', width=800, color='kind')
fig.show()